In [5]:
import argparse
import networkx as nx
import torch
import torch.nn as nn

!pip install torch-geometric
from torch_geometric.nn import GCNConv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.9 MB/s eta 0:00:00a 0:00:01


TODO:
Implement teacher network architecture and training along with flags for datasets and teacher model architecture (Chanikya and Nithin)
eg: python3 train_teacher.py --dataset=cora --model=SAGE --epochs-100 --lr=0.01 . Add flags for other hyperparameters if necessary (Chanikya and Nithin)
Other teacher model architectures - GCN, GAT, APPNP (Chanikya and Nithin + others based on availability)

In [6]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='./Cora', name='Cora')

data = dataset[0]
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

Processing...


Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7


Done!


In [7]:
# data.edge_index.t()

In [8]:
def load_data(dataset):
    if dataset == "cora":
        dataset = Planetoid(root='./Cora', name='Cora')
        data = dataset[0]
        graph_nx = nx.Graph()
        graph_nx.add_edges_from(data.edge_index.t().tolist())

        # Adding self-loops
        # graph_nx.add_edges_from((n, n) for n in graph_nx.nodes())
        
        # adj_tensor = torch.tensor(nx.to_numpy_array(graph_nx), dtype=torch.float).to('cuda')
        features = data.x
        labels = data.y

        train_idx = data.train_mask.nonzero(as_tuple=True)[0]
        val_idx = data.val_mask.nonzero(as_tuple=True)[0]
        test_idx = data.test_mask.nonzero(as_tuple=True)[0]
        
        return data.edge_index, features, labels, data.train_mask, data.val_mask, data.test_mask

In [9]:
cnt = 0
140+1000+500
2708
for x in data.val_mask:
    cnt += (x==True)
print(cnt)

tensor(500)


GCN => num layers, hidden, input dim, out, dp, activa,  

In [28]:
class GCN(nn.Module):
    def __init__(
        self,
        num_layers,
        input_dim,
        hidden_dim,
        output_dim,
        dropout_ratio,
        activation,
        norm_type="none"
    ):
        super().__init__()
        self.num_layers = num_layers
        self.norm_type = norm_type
        self.dropout = nn.Dropout(dropout_ratio)
        self.activation = activation

        self.layers = nn.ModuleList()
        self.norms = nn.ModuleList()

        if num_layers == 1:
            self.layers.append(GCNConv(input_dim, output_dim))
        else:
            self.layers.append(GCNConv(input_dim, hidden_dim))
            if norm_type == "batch":
                self.norms.append(nn.BatchNorm1d(hidden_dim))
            elif norm_type == "layer":
                self.norms.append(nn.LayerNorm(hidden_dim))

            for _ in range(num_layers - 2):
                self.layers.append(GCNConv(hidden_dim, hidden_dim))
                if norm_type == "batch":
                    self.norms.append(nn.BatchNorm1d(hidden_dim))
                elif norm_type == "layer":
                    self.norms.append(nn.LayerNorm(hidden_dim))

            self.layers.append(GCNConv(hidden_dim, output_dim))

    def forward(self, x, edge_index):
        h_list = []
        h = x
        for l, layer in enumerate(self.layers):
            h = layer(h, edge_index)
            if l != self.num_layers - 1:
                if self.norm_type != "none":
                    h = self.norms[l](h)
                h = self.activation(h)
                h = self.dropout(h)
                h_list.append(h)
        return h

In [30]:
def train(model, data, edge_index, labels, train_idx, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    out = model(data, edge_index)
    loss = criterion(out[train_idx], labels[train_idx])
    loss.backward()
    optimizer.step()
    return loss.item()

@torch.no_grad()
def evaluate(model, data, edge_index, labels, idx):
    model.eval()
    out = model(data, edge_index)
    pred = out[idx].argmax(dim=1)
    correct = (pred == labels[idx]).sum().item()
    acc = correct / sum(idx)
    return acc

In [39]:
edge_index, features, labels, train_idx, val_idx, test_idx = load_data("cora")
model = GCN(
    num_layers=3,
    input_dim=dataset.num_node_features,
    hidden_dim=64,
    output_dim=dataset.num_classes,
    dropout_ratio=0.8,
    activation=nn.functional.relu,
    norm_type="batch"
)
# model = GCN1(dataset.num_node_features, 64, dataset.num_classes)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(1, 150):
    loss = train(model, features, edge_index, labels, train_idx, optimizer, criterion)
    val_acc = evaluate(model, features, edge_index, labels, val_idx)
    if epoch % 10 == 0 or epoch == 1:
        test_acc = evaluate(model, features, edge_index, labels, test_idx)
        print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | Val Acc: {val_acc:.4f} | Test Acc: {test_acc:.4f}")


Epoch 001 | Loss: 2.2817 | Val Acc: 0.4040 | Test Acc: 0.3970
Epoch 010 | Loss: 0.7035 | Val Acc: 0.7620 | Test Acc: 0.7750
Epoch 020 | Loss: 0.3302 | Val Acc: 0.7580 | Test Acc: 0.7720
Epoch 030 | Loss: 0.1283 | Val Acc: 0.7500 | Test Acc: 0.7770
Epoch 040 | Loss: 0.1200 | Val Acc: 0.7500 | Test Acc: 0.7720
Epoch 050 | Loss: 0.0830 | Val Acc: 0.7440 | Test Acc: 0.7710
Epoch 060 | Loss: 0.0759 | Val Acc: 0.7580 | Test Acc: 0.7800
Epoch 070 | Loss: 0.0625 | Val Acc: 0.7540 | Test Acc: 0.7770
Epoch 080 | Loss: 0.0946 | Val Acc: 0.7520 | Test Acc: 0.7830
Epoch 090 | Loss: 0.0331 | Val Acc: 0.7520 | Test Acc: 0.7740
Epoch 100 | Loss: 0.0359 | Val Acc: 0.7440 | Test Acc: 0.7650
Epoch 110 | Loss: 0.0450 | Val Acc: 0.7480 | Test Acc: 0.7640
Epoch 120 | Loss: 0.0718 | Val Acc: 0.7360 | Test Acc: 0.7670
Epoch 130 | Loss: 0.0740 | Val Acc: 0.7380 | Test Acc: 0.7620
Epoch 140 | Loss: 0.0403 | Val Acc: 0.7380 | Test Acc: 0.7670


In [ ]:
model(data.x, data.edge_index)

In [2]:
class Teacher:
    def __init__(self, args):
        self.args = args
        pass
    def graph_split():
        pass
    def train_transductive():
        pass
    def train_inductive():
        pass
    

In [3]:
def main():
    parser = argparse.ArgumentParser(description="Teacher implementation")
    parser.add_argument('--num_runs', type=int, default=1, help='Number of runs')
    parser.add_argument('--setting', type=str, choices=['trans', 'ind'], required=True, help='Setting type: trans or ind')
    parser.add_argument('--data_path', type=str, required=True, help='Path to the dataset')
    parser.add_argument('--model_name', type=str, default='SAGE', help='Name of the model(SAGE, GCN, GAT, APPNP)')
    parser.add_argument('--num_layers', type=int, default=2, help='Number of layers in the model')
    parser.add_argument('--hidden_dim', type=int, default=128, help='Hidden dimension size')
    parser.add_argument('--drop_out', type=float, default=0, help='Dropout rate')
    parser.add_argument('--batch_sz', type=int, default=512, help='Batch size')
    parser.add_argument('--learning_rate', type=float, default=0.01, help='Learning rate')
    parser.add_argument('--output_path', type=str, default='./output', help='Path to save output')
    
    args = parser.parse_args()

    